In [1]:
import json
import requests
from keras import optimizers
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
import datetime
import xgboost as xgb
import joblib

sns.set_palette('Set2')
%matplotlib inline

Using TensorFlow backend.


In [2]:
df=pd.read_csv('../data/input_data/bitfinix.csv')
df.time = pd.to_datetime(df.time, unit='ms')

In [3]:
df.fillna(0,inplace=True)

In [4]:
split_date = pd.Timestamp(datetime.date(2019,1,1))

In [5]:
train_test_data = df[(pd.to_datetime(df["time"]) < split_date)]

In [6]:
validation_data = df[(pd.to_datetime(df["time"]) > split_date)]

In [7]:
split_row = len(train_test_data) - int(0.2 * len(train_test_data))
train_data = train_test_data.iloc[:split_row]
test_data = train_test_data.iloc[split_row:]

In [8]:
x_train = train_data[['high','low','open','volume']]
y_train = train_data[['close']]
x_test = test_data[['high','low','open','volume']]
y_test = test_data[['close']]

In [9]:
xgb_model = xgb.XGBRegressor(n_estimators=1000,
                             max_depth= 100,
                             max_features= 4,
                             min_samples_split = 2) 
xgb_model.fit(x_train, y_train);


[21:53:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
import joblib
filename = 'xgboost.sav'
joblib.dump(xgb_model,filename)

In [15]:
xgboostModel = joblib.load('xgboost.sav')

[12:08:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [16]:
xgboostModel

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.9, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=nan, n_estimators=10000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             silent=None, subsample=1, verbosity=1)

In [17]:
predicted_y = xgboostModel.predict(x_test)

In [18]:
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
print("Root Mean Squared Error(RMSE) : ", sqrt(mean_squared_error(y_test,predicted_y)))
print("Mean Absolute Error(MAE) : ", mean_absolute_error(y_test,predicted_y))

Root Mean Squared Error(RMSE) :  4.926249996936177
Mean Absolute Error(MAE) :  3.177571926815739
